In [8]:
# Reload the module to pick up any changes
import sys
import importlib
if 'polymarketAPI' in sys.modules:
    import polymarketAPI
    import polymarketAPI.client
    import polymarketAPI.api_builder
    importlib.reload(polymarketAPI.api_builder)
    importlib.reload(polymarketAPI.client)
    importlib.reload(polymarketAPI)
print("✅ Module reloaded")

✅ Module reloaded


In [11]:
import polars as pl
from polymarketAPI import PolymarketClient
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display

# Use the client to fetch all data
print("Fetching data from Polymarket API...")
client = PolymarketClient()
# Fetch data from each endpoint independently
df_leaderboard = client.fetch_leaderboard(limit=25)
df_closed_positions = client.fetch_closed_positions(limit=100)
df_trades = client.fetch_trades(limit=100)
df_activity = client.fetch_activity(limit=100)

# Rename columns for backward compatibility
if 'proxyWallet' in df_leaderboard.columns:
    df_leaderboard = df_leaderboard.rename({'proxyWallet': 'account', 'vol': 'volume'})
if 'proxyWallet' in df_closed_positions.columns:
    df_closed_positions = df_closed_positions.rename({'proxyWallet': 'account'})
if 'proxyWallet' in df_trades.columns:
    df_trades = df_trades.rename({'proxyWallet': 'maker'})
if 'proxyWallet' in df_activity.columns:
    df_activity = df_activity.rename({'proxyWallet': 'account'})

print("\n=== Leaderboard ===")
print(df_leaderboard.head())
print(f"\nLeaderboard shape: {df_leaderboard.shape}")

print("\n=== Closed Positions ===")
print(df_closed_positions.head())
print(f"\nClosed Positions shape: {df_closed_positions.shape}")

print("\n=== Trades ===")
print(df_trades.head())
print(f"\nTrades shape: {df_trades.shape}")

print("\n=== Activity ===")
print(df_activity.head())
print(f"\nActivity shape: {df_activity.shape}")

Fetching data from Polymarket API...

=== Leaderboard ===
shape: (5, 8)
┌──────┬─────────────┬─────────────┬───────────┬────────────┬────────────┬────────────┬────────────┐
│ rank ┆ account     ┆ userName    ┆ xUsername ┆ verifiedBa ┆ volume     ┆ pnl        ┆ profileIma │
│ ---  ┆ ---         ┆ ---         ┆ ---       ┆ dge        ┆ ---        ┆ ---        ┆ ge         │
│ str  ┆ str         ┆ str         ┆ str       ┆ ---        ┆ f64        ┆ f64        ┆ ---        │
│      ┆             ┆             ┆           ┆ bool       ┆            ┆            ┆ str        │
╞══════╪═════════════╪═════════════╪═══════════╪════════════╪════════════╪════════════╪════════════╡
│ 1    ┆ 0x492442eab ┆ 0x492442EaB ┆           ┆ false      ┆ 1.0987e6   ┆ 486751.857 ┆            │
│      ┆ 586f242b53b ┆ 586F242B53b ┆           ┆            ┆            ┆ 792        ┆            │
│      ┆ da933fd5…   ┆ Da933fD5…   ┆           ┆            ┆            ┆            ┆            │
│ 2    ┆ 0xe90bec87

In [12]:
# Visualize Leaderboard - Top Traders by PNL
if len(df_leaderboard) > 0 and 'pnl' in df_leaderboard.columns:
    fig1 = px.bar(
        df_leaderboard.head(15),
        x='account' if 'account' in df_leaderboard.columns else df_leaderboard.columns[0],
        y='pnl',
        title='Top 15 Traders by Daily PNL',
        labels={'pnl': 'Profit/Loss', 'account': 'Trader Address'},
        color='pnl',
        color_continuous_scale='RdYlGn'
    )
    fig1.update_layout(xaxis_tickangle=-45, height=500)
    display(fig1)

# Visualize Closed Positions - Best Realized PNL
if len(df_closed_positions) > 0 and 'realizedPnl' in df_closed_positions.columns:
    fig2 = px.bar(
        df_closed_positions.head(20),
        x=df_closed_positions.columns[0],
        y='realizedPnl',
        title='Top 20 Closed Positions by Realized PNL',
        labels={'realizedPnl': 'Realized P&L'},
        color='realizedPnl',
        color_continuous_scale='RdYlGn'
    )
    fig2.update_layout(xaxis_tickangle=-45, height=500)
    display(fig2)

# Visualize Trade Volume Distribution
if len(df_trades) > 0:
    if 'size' in df_trades.columns and 'price' in df_trades.columns:
        # Calculate trade value
        df_trades_viz = df_trades.with_columns([
            (pl.col('size').cast(pl.Float64) * pl.col('price').cast(pl.Float64)).alias('trade_value')
        ])
        
        fig3 = px.histogram(
            df_trades_viz,
            x='trade_value',
            nbins=30,
            title='Trade Value Distribution',
            labels={'trade_value': 'Trade Value (USD)'}
        )
        display(fig3)

# Activity Timeline
if len(df_activity) > 0 and 'timestamp' in df_activity.columns:
    df_activity_viz = df_activity.with_columns([
        pl.col('timestamp').cast(pl.Int64).alias('ts_int')
    ]).sort('ts_int')
    
    fig4 = px.scatter(
        df_activity_viz.head(100),
        x='timestamp',
        y=df_activity_viz.columns[1] if len(df_activity_viz.columns) > 1 else 'timestamp',
        title='Recent Activity Timeline (Last 100 Events)',
        labels={'timestamp': 'Time'}
    )
    fig4.update_layout(height=400)
    display(fig4)

print("\n✅ Visualization complete!")


✅ Visualization complete!


In [13]:
# Create a combined summary DataFrame with trader performance metrics
summary_data = []

if len(df_leaderboard) > 0:
    for row in df_leaderboard.iter_rows(named=True):
        trader_address = row.get('account', 'Unknown')
        
        # Get trader's closed positions if available
        trader_positions = []
        if len(df_closed_positions) > 0 and 'account' in df_closed_positions.columns:
            trader_positions = df_closed_positions.filter(pl.col('account') == trader_address)
        
        # Get trader's trades if available
        trader_trades = []
        if len(df_trades) > 0 and 'maker' in df_trades.columns:
            trader_trades = df_trades.filter(pl.col('maker') == trader_address)
        
        summary_data.append({
            'trader': trader_address[:10] + '...' if len(trader_address) > 10 else trader_address,
            'daily_pnl': row.get('pnl', 0),
            'volume': row.get('volume', 0),
            'closed_positions_count': len(trader_positions) if len(trader_positions) > 0 else 0,
            'trades_count': len(trader_trades) if len(trader_trades) > 0 else 0,
            'rank': row.get('rank', 0)
        })

# Create summary DataFrame
df_summary = pl.DataFrame(summary_data)
print("\n=== Top Traders Summary ===")
print(df_summary)

# Visualize summary with multiple metrics
if len(df_summary) > 0:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Daily PNL by Trader', 'Trading Volume', 
                       'Number of Closed Positions', 'Number of Trades'),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )
    
    top_n = min(10, len(df_summary))
    df_top = df_summary.head(top_n)
    
    # PNL
    fig.add_trace(
        go.Bar(x=df_top['trader'].to_list(), y=df_top['daily_pnl'].to_list(), 
               name='Daily PNL', marker_color='lightblue'),
        row=1, col=1
    )
    
    # Volume
    fig.add_trace(
        go.Bar(x=df_top['trader'].to_list(), y=df_top['volume'].to_list(), 
               name='Volume', marker_color='lightgreen'),
        row=1, col=2
    )
    
    # Closed Positions
    fig.add_trace(
        go.Bar(x=df_top['trader'].to_list(), y=df_top['closed_positions_count'].to_list(), 
               name='Positions', marker_color='lightyellow'),
        row=2, col=1
    )
    
    # Trades
    fig.add_trace(
        go.Bar(x=df_top['trader'].to_list(), y=df_top['trades_count'].to_list(), 
               name='Trades', marker_color='lightcoral'),
        row=2, col=2
    )
    
    fig.update_xaxes(tickangle=-45)
    fig.update_layout(height=800, showlegend=False, title_text="Top Traders Performance Dashboard")
    display(fig)

# Return the main dataframes for further analysis
print("\n✅ All dataframes created:")
print("- df_leaderboard: Top traders leaderboard")
print("- df_closed_positions: Historical closed positions")
print("- df_trades: Recent trades")
print("- df_activity: Recent activity")
print("- df_summary: Combined summary of top traders")


=== Top Traders Summary ===
shape: (25, 6)
┌───────────────┬───────────────┬───────────────┬────────────────────────┬──────────────┬──────┐
│ trader        ┆ daily_pnl     ┆ volume        ┆ closed_positions_count ┆ trades_count ┆ rank │
│ ---           ┆ ---           ┆ ---           ┆ ---                    ┆ ---          ┆ ---  │
│ str           ┆ f64           ┆ f64           ┆ i64                    ┆ i64          ┆ str  │
╞═══════════════╪═══════════════╪═══════════════╪════════════════════════╪══════════════╪══════╡
│ 0x492442ea... ┆ 486751.857792 ┆ 1.0987e6      ┆ 20                     ┆ 0            ┆ 1    │
│ 0xe90bec87... ┆ 198814.009903 ┆ 1.7566e6      ┆ 20                     ┆ 0            ┆ 2    │
│ 0xdb27bf2a... ┆ 156273.929034 ┆ 425569.047236 ┆ 20                     ┆ 0            ┆ 3    │
│ 0x1ff26f9f... ┆ 146750.854042 ┆ 380898.611817 ┆ 20                     ┆ 0            ┆ 4    │
│ 0x1bc0d88c... ┆ 126224.259473 ┆ 182202.749537 ┆ 15                     ┆ 0       


✅ All dataframes created:
- df_leaderboard: Top traders leaderboard
- df_closed_positions: Historical closed positions
- df_trades: Recent trades
- df_activity: Recent activity
- df_summary: Combined summary of top traders


## ✅ Analysis Complete!

The notebook has successfully:
1. ✅ Fetched data from all 4 Polymarket API endpoints
2. ✅ Created Polars DataFrames for analysis
3. ✅ Generated visualizations showing:
   - Top traders by daily PNL
   - Best closed positions by realized PNL
   - Trade value distribution
   - Recent activity timeline
   - Combined performance dashboard

All dataframes are available for further analysis:
- `df_leaderboard` - Top traders leaderboard
- `df_closed_positions` - Historical closed positions
- `df_trades` - Recent trades  
- `df_activity` - Recent activity
- `df_summary` - Combined trader summary with metrics